This method uses the content based approach to include the user's genre preference and recommends movies  similar to user's highly rated movies.

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [50]:
# Load movie data
movies = pd.read_csv("../dataset/tmdb_movies.csv")#, converters={"genres": literal_eval, "tag": literal_eval})

In [51]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [52]:
movies.shape

(4803, 20)

In [53]:
movies['tagline'] = movies['tagline'].fillna('')
movies['description'] = movies['overview'] + movies['tagline']
movies['description'] = movies['description'].fillna('')

In [54]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['description'])

In [55]:
tfidf_matrix.shape

(4803, 149317)

In [56]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [57]:
movies = movies.reset_index()
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])
indices.head(2)

title
Avatar                                      0
Pirates of the Caribbean: At World's End    1
dtype: int64

In [58]:
def get_recommendations(title):
    idx = indices[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

The title 'Doctor Who: Last Christmas' is not found in the indices dictionary.
No recommendations available.


In [59]:
get_recommendations('The Dark Knight').head(45)

TypeError: 'NoneType' object is not callable

In [ ]:
get_recommendations('The Godfather').head(10)

2731     The Godfather: Part II
1873                 Blood Ties
3859            Lords of London
3623                       Made
3727                 Easy Money
3164              Loose Cannons
1475          Playing for Keeps
3960               The Deported
2038              Summer of Sam
867     The Godfather: Part III
Name: title, dtype: object

In [ ]:
get_recommendations('Toy Story').head(10)

343                Toy Story 2
42                 Toy Story 3
1779    The 40 Year Old Virgin
891            Man on the Moon
3379              Factory Girl
3873             Class of 1984
2869    For Your Consideration
3065                Heartbeeps
3383                 Losin' It
2569               Match Point
Name: title, dtype: object

In [19]:
get_recommendations('Jumanji').head(10)

KeyError: 'Jumanji'

In [ ]:
get_recommendations('Doctor Who: Last Christmas').head(10)

The title 'Doctor Who: Last Christmas' is not found in the indices dictionary.


AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
movies[movies['title'] == 'Doctor Who: Last Christmas']['genres'].values[0]

"['Adventure', 'Drama', 'Fantasy', 'Sci-Fi']"

In [ ]:
movies[movies['title'] == 'Doctor Who: Last Christmas']['overview'].values[0]

'The Doctor and Clara face their Last Christmas. Trapped on an Arctic base, under attack from terrifying creatures, who are you going to call? Santa Claus!'

In [ ]:
get_recommendations('Inception').head(10)

49042                   III
7508                 Cypher
160                   Crumb
10983           Renaissance
9150              The Brave
26142    Closer to the Moon
30806         Dear Murderer
34372              Hollywoo
22995      The Monkey's Paw
20621        A Woman's Face
Name: title, dtype: object

In [ ]:
movies[movies['title'] == 'III']['overview'].values[0]

"A small European town, where sisters Ayia and Mirra live, gets struck down by an unknown disease which takes many lives. Following their mother's death, the younger sister falls ill. Having realized that conventional medicine is useless in the face of the sister's disease, Ayia seeks help from Father Herman, a parish priest and a close family friend. In his house she finds books that are very far from the conventional religion. She gets to know that only penetration into Mirra's sick subconscious mind and discovery of the true cause of her disease will give her a chance to save her sister. Ayia is ready to go through this terrifying ritual, dive into the depths of the subconscious mind, and face the demons residing there."

In [ ]:
movies[movies['title'] == 'Inception']['overview'].values[0]

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.'

In [ ]:
popularity_df = movies[['popularity', 'vote_average', 'vote_count']]
popularity_df.corr()

,popularity,vote_average,vote_count
popularity,1.000000,0.154253,0.559918
vote_average,0.154253,1.000000,0.123681
vote_count,0.559918,0.123681,1.000000


In [ ]:
# pd.plotting.scatter_matrix(popularity_df, figsize=(10, 10))
# plt.show()

## Include genre in TF-IDF

In [ ]:
movies['tagline'] = movies['tagline'].fillna('')
movies['description_genre'] = movies['overview'] + movies['tagline'] + 2*movies['genres']
movies['description_genre'] = movies['description_genre'].fillna('')

In [ ]:
tf_new = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix_new = tf_new.fit_transform(movies['description_genre'])

In [ ]:
cosine_sim_new = linear_kernel(tfidf_matrix_new, tfidf_matrix_new)

In [ ]:
tf_new.vocabulary_['scifi']

873101

In [ ]:
movies = movies.reset_index()
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])
indices.head(2)

title
Toy Story    0
Jumanji      1
dtype: int64

In [ ]:
def get_recommendations_new(title):
    idx = indices[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
get_recommendations_new('Doctor Who: Last Christmas').head(10)

30647                         20 Years After
19000                 4:44 Last Day on Earth
38193    Wizards of Waverly Place: The Movie
36355                     Under the Mountain
10418                         Born in Flames
45510                              Team Thor
28                 The City of Lost Children
20474              It's Such a Beautiful Day
27268                    Master of the World
40678                           On the Comet
Name: title, dtype: object

In [ ]:
movies[movies['title'] == 'Doctor Who: Last Christmas']['genres'].values

array(["['Adventure', 'Drama', 'Fantasy', 'Sci-Fi']"], dtype=object)

In [ ]:
movies[movies['title'] == "It's Such a Beautiful Day"]['genres'].values

array(["['Animation', 'Comedy', 'Drama', 'Fantasy', 'Sci-Fi']",
       "['Animation', 'Comedy', 'Drama']"], dtype=object)

In [ ]:
movies[movies['title'] == 'Doctor Who: Last Christmas']['overview'].values

array(['The Doctor and Clara face their Last Christmas. Trapped on an Arctic base, under attack from terrifying creatures, who are you going to call? Santa Claus!'],
      dtype=object)

In [ ]:
movies[movies['title'] == '4:44 Last Day on Earth']['overview'].values

array(['A look at how a painter and a successful actor spend their last day together before the world comes to an end.'],
      dtype=object)

In [ ]:
movies[movies['title'] == '20 Years After']['genres'].values

array(["['Drama', 'Fantasy', 'Sci-Fi']"], dtype=object)

In [ ]:
movies[movies['title'] == '20 Years After']['overview'].values

array(['In the middle of nowhere, 20 years after an apocalyptic terrorist event that obliterated the face of the world!'],
      dtype=object)

In [ ]:
get_recommendations_new('Inception').head(10)

9278            Possible Worlds
12901                  Watchmen
26568                 Implanted
21110                Iron Man 3
6826     The Matrix Revolutions
43137                Wild Palms
9504                 The Jacket
27816         Talking to Heaven
44280     Kiss Kiss (Bang Bang)
19470                  The Door
Name: title, dtype: object

In [ ]:
movies[movies['title'] == 'Game Over']['description_genre'].values

array(["...['Crime', 'Drama', 'Thriller']['Crime', 'Drama', 'Thriller']",
       "Short depicting the endings of various classic video games in a whimsical fashion.['Drama']['Drama']"],
      dtype=object)

In [ ]:
movies[movies['title'] == 'Inception']['description_genre'].values

array(['Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.Your mind is the scene of the crime.[\'Action\', \'Crime\', \'Drama\', \'Mystery\', \'Sci-Fi\', \'Thriller\', \'IMAX\'][\'Action\', \'Crime\', \'Drama\', \'Mystery\', \'Sci-Fi\', \'Thriller\', \'IMAX\']'],
      dtype=object)

In [ ]:
get_recommendations_new('Avatar').head(10)

13931    Transformers: Revenge of the Fallen
22239                           Ender's Game
18758       Journey 2: The Mysterious Island
48131                              Megafault
49100      Abraxas, Guardian of the Universe
11142                       Superman Returns
18006                           The Avengers
16436                           TRON: Legacy
16955                        Mars Needs Moms
27564           Star Wars: The Force Awakens
Name: title, dtype: object

### IMDB Weighted Average 

In [ ]:
# this is V
vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')

# this is R
vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')

# this is C
C = vote_averages.mean()
C

5.246875482178676

In [ ]:
m = vote_counts.quantile(0.95)
m

434.0

In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

## Improved Recommendations

In [ ]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies_x = movies.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies_x[movies_x['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies_x[movies_x['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies_x[(movies_x['vote_count'] >= m) & (movies_x['vote_count'].notnull()) &
                       (movies_x['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [ ]:
improved_recommendations('Doctor Who: Last Christmas').head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,title,vote_count,vote_average,wr
28,The City of Lost Children,308,7,5.974588
38193,Wizards of Waverly Place: The Movie,323,6,5.568222
20474,It's Such a Beautiful Day,94,7,5.558985
45510,Team Thor,93,7,5.556250
22106,After the Dark,283,6,5.544134
46593,Marvel One-Shot: A Funny Thing Happened on the...,72,7,5.496332
7521,Frank Herbert's Dune,114,6,5.403547
9410,Immortal,131,5,5.189635
23179,The Zero Theorem,383,5,5.131143
19000,4:44 Last Day on Earth,48,4,5.122705


In [ ]:
improved_recommendations('Avatar').head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,title,vote_count,vote_average,wr
18006,The Avengers,12000,7,6.938808
27564,Star Wars: The Force Awakens,7993,7,6.909712
13738,Star Trek,4574,7,6.848072
21207,Star Trek Into Darkness,4479,7,6.845134
21356,Man of Steel,6462,6,5.952602
21028,Oblivion,4862,6,5.938282
7957,Spider-Man 2,4432,6,5.932829
23811,Godzilla,3232,6,5.910841
13931,Transformers: Revenge of the Fallen,3192,6,5.909858
6826,The Matrix Revolutions,3155,6,5.908928
